# Project notebook

The following notebook is an excerpt and re-written example from a _real_ production model.

The overall purpose of the ML algorithm is to identify users on the website that are new possible customers. This is done by collecting behaviour data from the users as input, and the target is whether they converted/turned into customers -- essentially a classification problem. 

This notebook only focuses on the data processing part. As you know, there are multiple steps in an ML pipeline, and it's not always they are neatly separated like this. For the exam project, they will not be, and that is part of the challenge for you. For production code, it should also not be Python notebooks since, as you may well see, it is difficult to work with and collaborate on them in an automated way.

There is a lot of "fluff" in such a notebook. This ranges from comments and markdown cells to commented out code and random print statements. That is not necessary in a properly managed project where you can use git to check the version history and such. 

What is important for you is the identify the entry points into the code and segment them out into easily understandable chunks. Additionally, you might want to follow some basic code standards, such as:

- Import only libraries in the beginning of the files
- Define functions in the top of the scripts, or if used multiple places, move into a util.py script or such
- Remove unused/commented out code
- Follow the [PEP 8](https://peps.python.org/pep-0008/) style guide (and others)
  
Another thing to note is that comments can be misleading. Even if the markdown cell or inline comments says it does _X_, don't be surprised if it actually does _Y_. Sometimes additional text can be a blessing, but it can also be a curse sometimes. Remember, though, that your task is to make sure the code runs as before after refactoring the notebook into other files, not update/improve the model or flow to reflect what the comments might say.

***

# DATA PROCESSING

In this section, we will perform Exploratory Data Analysis (EDA) to better understand the dataset before proceeding with more advanced analysis. EDA helps us get a sense of the data’s structure, identify patterns, and spot any potential issues like missing values or outliers. By doing so, we can gain a clearer understanding of the data's key characteristics.

We will start with summary statistics to review basic measures like mean, median, and variance, providing an initial overview of the data distribution. Then, we’ll create visualizations such as histograms, box plots, and scatter plots to explore relationships between variables, check for any skewness, and highlight outliers.

The purpose of this EDA is to ensure that the dataset is clean and well-structured for further analysis. This step also helps us identify any necessary data transformations and informs decisions on which features might be most relevant for modeling in later stages.

# Create artifact directory
We want to create a directory for storing all the artifacts in the current directory. Users can load all the artifacts later for data cleaning pipelines and inferencing.

In [1]:
# dbutils.widgets.text("Training data max date", "2024-01-31")
# dbutils.widgets.text("Training data min date", "2024-01-01")
# max_date = dbutils.widgets.get("Training data max date")
# min_date = dbutils.widgets.get("Training data min date")

# testnng
max_date = "2024-01-31"
min_date = "2024-01-01"

In [2]:
import os
import shutil
from pprint import pprint

# shutil.rmtree("./artifacts",ignore_errors=True)
os.makedirs("artifacts",exist_ok=True)
print("Created artifacts directory")

Created artifacts directory


# Pandas dataframe print options

In [3]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format',lambda x: "%.3f" % x)

# Helper functions

* **describe_numeric_col**: Calculates various descriptive stats for a numeric column in a dataframe.
* **impute_missing_values**: Imputes the mean/median for numeric columns or the mode for other types.

In [4]:
def describe_numeric_col(x):
    """
    Parameters:
        x (pd.Series): Pandas col to describe.
    Output:
        y (pd.Series): Pandas series with descriptive stats. 
    """
    return pd.Series(
        [x.count(), x.isnull().count(), x.mean(), x.min(), x.max()],
        index=["Count", "Missing", "Mean", "Min", "Max"]
    )

def impute_missing_values(x, method="mean"):
    """
    Parameters:
        x (pd.Series): Pandas col to describe.
        method (str): Values: "mean", "median"
    """
    if (x.dtype == "float64") | (x.dtype == "int64"):
        x = x.fillna(x.mean()) if method=="mean" else x.fillna(x.median())
    else:
        x = x.fillna(x.mode()[0])
    return x

# Read data

We read the latest data from our data lake source. Here we load it locally after having pulled it from DVC.

In [5]:
!dvc pull

'dvc' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
print("Loading training data")

data = pd.read_csv("./artifacts/raw_data.csv")

print("Total rows:", data.count())
display(data.head(5))


Loading training data
Total rows: lead_id                              12345
lead_indicator                       11753
date_part                            12345
is_active                            12345
marketing_consent                    12345
first_booking                        12345
existing_customer                    12345
last_seen                            12345
source                               12345
domain                               12345
country                              12345
visited_learn_more_before_booking    12345
visited_faq                          12345
purchases                            12345
time_spent                           12345
customer_group                       12345
onboarding                           12345
customer_code                        12294
n_visits                             12345
dtype: int64


,lead_id,lead_indicator,date_part,is_active,marketing_consent,first_booking,existing_customer,last_seen,source,domain,country,visited_learn_more_before_booking,visited_faq,purchases,time_spent,customer_group,onboarding,customer_code,n_visits
0,0,0.000,2024-01-24,0,True,2024-01-25,False,2024-01-13,organic,.dk,US,8,10,5,115.064,2,True,NaN,9
1,1,NaN,2024-01-1,1,True,2024-01-24,False,2024-01-6,signup,.dk,US,0,3,5,119.907,2,False,AGMVEYWACO,1
2,2,1.000,2024-01-27,0,True,2024-01-30,True,2024-01-30,signup,.dk,DK,7,10,3,100.474,5,False,STWPXVNOWA,16
3,3,1.000,2024-01-28,0,False,2024-01-10,False,2024-01-11,li,.cn,US,10,10,4,98.572,4,True,HZOKZERLZD,16
4,4,0.000,2024-01-5,0,True,2024-01-30,True,2024-01-25,fb,.com,US,5,1,5,101.996,9,False,CVLIHCAPZN,2


In [8]:
import pandas as pd
import datetime
import json

if not max_date:
    max_date = pd.to_datetime(datetime.datetime.now().date()).date()
else:
    max_date = pd.to_datetime(max_date).date()

min_date = pd.to_datetime(min_date).date()

# Time limit data
data["date_part"] = pd.to_datetime(data["date_part"]).dt.date
data = data[(data["date_part"] >= min_date) & (data["date_part"] <= max_date)]

min_date = data["date_part"].min()
max_date = data["date_part"].max()
date_limits = {"min_date": str(min_date), "max_date": str(max_date)}
with open("./artifacts/date_limits.json", "w") as f:
    json.dump(date_limits, f)

# Feature selection

Not all columns are relevant for modelling

In [9]:
data = data.drop(
    [
        "is_active", "marketing_consent", "first_booking", "existing_customer", "last_seen"
    ],
    axis=1
)

In [10]:
#Removing columns that will be added back after the EDA
data = data.drop(
    ["domain", "country", "visited_learn_more_before_booking", "visited_faq"],
    axis=1
)

# Data cleaning
* Remove rows with empty target variable
* Remove rows with other invalid column data

In [11]:
import numpy as np

data["lead_indicator"].replace("", np.nan, inplace=True)
data["lead_id"].replace("", np.nan, inplace=True)
data["customer_code"].replace("", np.nan, inplace=True)

data = data.dropna(axis=0, subset=["lead_indicator"])
data = data.dropna(axis=0, subset=["lead_id"])

data = data[data.source == "signup"]
result=data.lead_indicator.value_counts(normalize = True)

print("Target value counter")
for val, n in zip(result.index, result):
    print(val, ": ", n)
data

Target value counter
0.0 :  0.5318352059925093
1.0 :  0.4681647940074906


,lead_id,lead_indicator,date_part,source,purchases,time_spent,customer_group,onboarding,customer_code,n_visits
2,2,1.000,2024-01-27,signup,3,100.474,5,False,STWPXVNOWA,16
10,10,0.000,2024-01-11,signup,8,105.899,4,True,JWLXPVANUM,4
13,13,0.000,2024-01-22,signup,6,86.717,6,True,WNKCTHUHYI,5
17,17,1.000,2024-01-07,signup,5,106.952,8,False,INIQMTGGOO,8
19,19,0.000,2024-01-20,signup,3,94.668,9,True,SJGDAANOYJ,20
...,...,...,...,...,...,...,...,...,...,...
12328,12328,1.000,2024-01-01,signup,2,98.999,5,True,AYAVVZAMGV,14
12330,12330,1.000,2024-01-14,signup,6,101.390,4,True,YFKCVZSAJF,18
12334,12334,0.000,2024-01-21,signup,1,93.840,5,True,ROVGLTVJNL,1
12337,12337,1.000,2024-01-03,signup,0,110.320,6,True,BGQNNVRTDH,18


# Create categorical data columns

In [12]:
vars = [
    "lead_id", "lead_indicator", "customer_group", "onboarding", "source", "customer_code"
]

for col in vars:
    data[col] = data[col].astype("object")
    print(f"Changed {col} to object type")

Changed lead_id to object type
Changed lead_indicator to object type
Changed customer_group to object type
Changed onboarding to object type
Changed source to object type
Changed customer_code to object type


# Separate categorical and continuous columns

In [13]:
cont_vars = data.loc[:, ((data.dtypes=="float64")|(data.dtypes=="int64"))]
cat_vars = data.loc[:, (data.dtypes=="object")]

print("\nContinuous columns: \n")
pprint(list(cont_vars.columns), indent=4)
print("\n Categorical columns: \n")
pprint(list(cat_vars.columns), indent=4)


Continuous columns: 

['purchases', 'time_spent', 'n_visits']

 Categorical columns: 

[   'lead_id',
    'lead_indicator',
    'date_part',
    'source',
    'customer_group',
    'onboarding',
    'customer_code']


# Outliers

Outliers are data points that significantly differ from the majority of observations in a dataset and can distort statistical analysis or model performance. To identify and remove outliers, one common method is to use the Z-score, which measures how many standard deviations a data point is from the mean. Data points with a Z-score greater than 2 (or sometimes 3) standard deviations away from the mean are typically considered outliers. By applying this threshold, we can filter out values that fall outside the normal range of the data, ensuring that the remaining dataset is more representative and less influenced by extreme values.

In [14]:
cont_vars = cont_vars.apply(lambda x: x.clip(lower = (x.mean()-2*x.std()),
                                             upper = (x.mean()+2*x.std())))
outlier_summary = cont_vars.apply(describe_numeric_col).T
outlier_summary.to_csv('./artifacts/outlier_summary.csv')
outlier_summary

,Count,Missing,Mean,Min,Max
purchases,2937.000,2937.000,5.000,0.578,9.495
time_spent,2937.000,2937.000,100.015,80.082,119.902
n_visits,2937.000,2937.000,8.884,1.000,22.334


# Impute data

In real-world datasets, missing data is a common occurrence due to various factors such as human error, incomplete data collection processes, or system failures. These gaps in the data can hinder analysis and lead to biased results if not properly addressed. Since many analytical and machine learning algorithms require complete data, handling missing values is an essential step in the data preprocessing phase.

In the next code block, we will handle missing data by performing imputation. For numerical columns, we will replace missing values with the mean or median of the entire column, which provides a reasonable estimate based on the existing data. For categorical columns (object type), we will use the mode, or most frequent value, to fill in missing entries. This approach helps us maintain a complete dataset while ensuring that the imputed values align with the general distribution of each column.

In [15]:
cat_missing_impute = cat_vars.mode(numeric_only=False, dropna=True)
cat_missing_impute.to_csv("./artifacts/cat_missing_impute.csv")
cat_missing_impute

,lead_id,lead_indicator,date_part,source,customer_group,onboarding,customer_code
0,2,0.000,2024-01-06,signup,5,True,AAAEJOLSFX
1,10,NaN,NaN,NaN,NaN,NaN,AACEDUWABX
2,13,NaN,NaN,NaN,NaN,NaN,AAJURHRBIM
3,17,NaN,NaN,NaN,NaN,NaN,AAMFOUYNWS
4,19,NaN,NaN,NaN,NaN,NaN,AAXKODEIPG
...,...,...,...,...,...,...,...
2932,12328,NaN,NaN,NaN,NaN,NaN,NaN
2933,12330,NaN,NaN,NaN,NaN,NaN,NaN
2934,12334,NaN,NaN,NaN,NaN,NaN,NaN
2935,12337,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Continuous variables missing values
cont_vars = cont_vars.apply(impute_missing_values)
cont_vars.apply(describe_numeric_col).T

,Count,Missing,Mean,Min,Max
purchases,2937.000,2937.000,5.000,0.578,9.495
time_spent,2937.000,2937.000,100.015,80.082,119.902
n_visits,2937.000,2937.000,8.884,1.000,22.334


In [17]:
cat_vars.loc[cat_vars['customer_code'].isna(),'customer_code'] = 'None'
cat_vars = cat_vars.apply(impute_missing_values)
cat_vars.apply(lambda x: pd.Series([x.count(), x.isnull().sum()], index = ['Count', 'Missing'])).T
cat_vars

,lead_id,lead_indicator,date_part,source,customer_group,onboarding,customer_code
2,2,1.000,2024-01-27,signup,5,False,STWPXVNOWA
10,10,0.000,2024-01-11,signup,4,True,JWLXPVANUM
13,13,0.000,2024-01-22,signup,6,True,WNKCTHUHYI
17,17,1.000,2024-01-07,signup,8,False,INIQMTGGOO
19,19,0.000,2024-01-20,signup,9,True,SJGDAANOYJ
...,...,...,...,...,...,...,...
12328,12328,1.000,2024-01-01,signup,5,True,AYAVVZAMGV
12330,12330,1.000,2024-01-14,signup,4,True,YFKCVZSAJF
12334,12334,0.000,2024-01-21,signup,5,True,ROVGLTVJNL
12337,12337,1.000,2024-01-03,signup,6,True,BGQNNVRTDH


# Data standardisation

Standardization, or scaling, becomes necessary when continuous independent variables are measured on different scales, as this can lead to unequal contributions to the analysis. The objective is to rescale these variables so they have comparable ranges and/or variances, ensuring a more balanced influence in the model.

In [18]:
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler_path = "./artifacts/scaler.pkl"

scaler = MinMaxScaler()
scaler.fit(cont_vars)

joblib.dump(value=scaler, filename=scaler_path)
print("Saved scaler in artifacts")

cont_vars = pd.DataFrame(scaler.transform(cont_vars), columns=cont_vars.columns)
cont_vars

Saved scaler in artifacts


,purchases,time_spent,n_visits
0,0.272,0.512,0.703
1,0.832,0.648,0.141
2,0.608,0.167,0.187
3,0.496,0.675,0.328
4,0.272,0.366,0.891
...,...,...,...
2932,0.159,0.475,0.609
2933,0.608,0.535,0.797
2934,0.047,0.346,0.000
2935,0.000,0.759,0.797


# Combine data

In [19]:
cont_vars = cont_vars.reset_index(drop=True)
cat_vars = cat_vars.reset_index(drop=True)
data = pd.concat([cat_vars, cont_vars], axis=1)
print(f"Data cleansed and combined.\nRows: {len(data)}")
data

Data cleansed and combined.
Rows: 2937


,lead_id,lead_indicator,date_part,source,customer_group,onboarding,customer_code,purchases,time_spent,n_visits
0,2,1.000,2024-01-27,signup,5,False,STWPXVNOWA,0.272,0.512,0.703
1,10,0.000,2024-01-11,signup,4,True,JWLXPVANUM,0.832,0.648,0.141
2,13,0.000,2024-01-22,signup,6,True,WNKCTHUHYI,0.608,0.167,0.187
3,17,1.000,2024-01-07,signup,8,False,INIQMTGGOO,0.496,0.675,0.328
4,19,0.000,2024-01-20,signup,9,True,SJGDAANOYJ,0.272,0.366,0.891
...,...,...,...,...,...,...,...,...,...,...
2932,12328,1.000,2024-01-01,signup,5,True,AYAVVZAMGV,0.159,0.475,0.609
2933,12330,1.000,2024-01-14,signup,4,True,YFKCVZSAJF,0.608,0.535,0.797
2934,12334,0.000,2024-01-21,signup,5,True,ROVGLTVJNL,0.047,0.346,0.000
2935,12337,1.000,2024-01-03,signup,6,True,BGQNNVRTDH,0.000,0.759,0.797


# Data drift artifact

In [20]:
import json

data_columns = list(data.columns)
with open('./artifacts/columns_drift.json','w+') as f:           
    json.dump(data_columns,f)
    
data.to_csv('./artifacts/training_data.csv', index=False)

# Binning object columns

In [ ]:
data.columns

In [21]:
data['bin_source'] = data['source']
values_list = ['li', 'organic','signup','fb']
data.loc[~data['source'].isin(values_list),'bin_source'] = 'Others'
mapping = {'li' : 'socials', 
           'fb' : 'socials', 
           'organic': 'group1', 
           'signup': 'group1'
           }

data['bin_source'] = data['source'].map(mapping)

# Save gold medallion dataset

In [ ]:
#spark.sql(f"drop table if exists train_gold")


In [22]:
# data_gold = spark.createDataFrame(data)
# data_gold.write.saveAsTable('train_gold')
# dbutils.notebook.exit(('training_golden_data',most_recent_date))

data.to_csv('./artifacts/train_data_gold.csv', index=False)

# MODEL TRAINING

Training the model uses a training dataset for training an ML algorithm. It has sample output data and the matching input data that affects the output.

In [23]:
import datetime

# Constants used:
current_date = datetime.datetime.now().strftime("%Y_%B_%d")
data_gold_path = "./artifacts/train_data_gold.csv"
data_version = "00000"
experiment_name = current_date

# Create paths

Maybe the artifacts path has not been created during data cleaning

In [24]:
import os
import shutil

os.makedirs("artifacts", exist_ok=True)
os.makedirs("mlruns", exist_ok=True)
os.makedirs("mlruns/.trash", exist_ok=True)

In [28]:
import mlflow

mlflow.set_experiment(experiment_name)

2024/11/27 10:52:34 INFO mlflow.tracking.fluent: Experiment with name '2024_November_27' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/Projects/SoftwareDev2024/itu-sdse-project-main/notebooks/mlruns/513283018559583915', creation_time=1732701154543, experiment_id='513283018559583915', last_update_time=1732701154543, lifecycle_stage='active', name='2024_November_27', tags={}>

# Helper functions

* *create_dummies*: Create one-hot encoding columns in the data.

In [29]:
def create_dummy_cols(df, col):
    df_dummies = pd.get_dummies(df[col], prefix=col, drop_first=True)
    new_df = pd.concat([df, df_dummies], axis=1)
    new_df = new_df.drop(col, axis=1)
    return new_df

# Load training data
We use the training data we cleaned earlier

In [30]:
data = pd.read_csv(data_gold_path)
print(f"Training data length: {len(data)}")
data.head(5)

Training data length: 2937


,lead_id,lead_indicator,date_part,source,customer_group,onboarding,customer_code,purchases,time_spent,n_visits,bin_source
0,2,1.000,2024-01-27,signup,5,False,STWPXVNOWA,0.272,0.512,0.703,group1
1,10,0.000,2024-01-11,signup,4,True,JWLXPVANUM,0.832,0.648,0.141,group1
2,13,0.000,2024-01-22,signup,6,True,WNKCTHUHYI,0.608,0.167,0.187,group1
3,17,1.000,2024-01-07,signup,8,False,INIQMTGGOO,0.496,0.675,0.328,group1
4,19,0.000,2024-01-20,signup,9,True,SJGDAANOYJ,0.272,0.366,0.891,group1


# Data type split

In [31]:
data = data.drop(["lead_id", "customer_code", "date_part"], axis=1)

cat_cols = ["customer_group", "onboarding", "bin_source", "source"]
cat_vars = data[cat_cols]

other_vars = data.drop(cat_cols, axis=1)

# Dummy variable for categorical vars

1. Create one-hot encoded cols for cat vars
2. Change to floats

In [32]:
import pandas as pd

for col in cat_vars:
    cat_vars[col] = cat_vars[col].astype("category")
    cat_vars = create_dummy_cols(cat_vars, col)

data = pd.concat([other_vars, cat_vars], axis=1)

for col in data:
    data[col] = data[col].astype("float64")
    print(f"Changed column {col} to float")

Changed column lead_indicator to float
Changed column purchases to float
Changed column time_spent to float
Changed column n_visits to float
Changed column customer_group_2 to float
Changed column customer_group_3 to float
Changed column customer_group_4 to float
Changed column customer_group_5 to float
Changed column customer_group_6 to float
Changed column customer_group_7 to float
Changed column customer_group_8 to float
Changed column customer_group_9 to float
Changed column onboarding_True to float


# Splitting data

In [33]:
y = data["lead_indicator"]
X = data.drop(["lead_indicator"], axis=1)

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.15, stratify=y
)
y_train

335    1.000
499    0.000
2850   1.000
2142   0.000
1019   0.000
        ... 
1378   0.000
2771   0.000
28     0.000
1410   1.000
2488   1.000
Name: lead_indicator, Length: 2496, dtype: float64

# Model training

This stage involves training the ML algorithm by providing it with datasets, where the learning process takes place. Consistent training can significantly enhance the model's prediction accuracy. It's essential to initialize the model's weights randomly so the algorithm can effectively learn to adjust them.

# XGBoost

In [36]:
from xgboost import XGBRFClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint

model = XGBRFClassifier(random_state=42)
params = {
    "learning_rate": uniform(1e-2, 3e-1),
    "min_split_loss": uniform(0, 10),
    "max_depth": randint(3, 10),
    "subsample": uniform(0, 1),
    "objective": ["reg:squarederror", "binary:logistic", "reg:logistic"],
    "eval_metric": ["aucpr", "error"]
}

model_grid = RandomizedSearchCV(model, param_distributions=params, n_jobs=-1, verbose=3, n_iter=10, cv=10)

model_grid.fit(X_train, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=XGBRFClassifier(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bytree=None, device=None,
                                             early_stopping_rounds=None,
                                             enable_categorical=False,
                                             eval_metric=None,
                                             feature_types=None, gamma=None,
                                             grow_policy=None,
                                             importance_type=None,
                                             interaction_constraints=None,
                                             max_bin=None,
                                             max_cat_threshold=N...
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000153E6FE9B50>,
                                        'min_split_loss': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000153E6EF8210>,
                                        'objective': ['reg:squarederror',
                                                      'binary:logistic',
                                                      'reg:logistic'],
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000153E6FEA3D0>},
                   verbose=3)

# Model test accuracy

In [37]:
from sklearn.metrics import accuracy_score

best_model_xgboost_params = model_grid.best_params_
print("Best xgboost params")
pprint(best_model_xgboost_params)

y_pred_train = model_grid.predict(X_train)
y_pred_test = model_grid.predict(X_test)
print("Accuracy train", accuracy_score(y_pred_train, y_train ))
print("Accuracy test", accuracy_score(y_pred_test, y_test))


Best xgboost params
{'eval_metric': 'error',
 'learning_rate': 0.22544853749905067,
 'max_depth': 4,
 'min_split_loss': 9.895401467931196,
 'objective': 'reg:squarederror',
 'subsample': 0.10327782105666328}
Accuracy train 0.8173076923076923
Accuracy test 0.8027210884353742


# XGBoost performance overview
* Confusion matrix
* Classification report

In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

conf_matrix = confusion_matrix(y_test, y_pred_test)
print("Test actual/predicted\n")
print(pd.crosstab(y_test, y_pred_test, rownames=['Actual'], colnames=['Predicted'], margins=True),'\n')
print("Classification report\n")
print(classification_report(y_test, y_pred_test),'\n')

conf_matrix = confusion_matrix(y_train, y_pred_train)
print("Train actual/predicted\n")
print(pd.crosstab(y_train, y_pred_train, rownames=['Actual'], colnames=['Predicted'], margins=True),'\n')
print("Classification report\n")
print(classification_report(y_train, y_pred_train),'\n')

Test actual/predicted

Predicted    0    1  All
Actual                  
0.0        205   30  235
1.0         57  149  206
All        262  179  441 

Classification report

              precision    recall  f1-score   support

         0.0       0.78      0.87      0.82       235
         1.0       0.83      0.72      0.77       206

    accuracy                           0.80       441
   macro avg       0.81      0.80      0.80       441
weighted avg       0.81      0.80      0.80       441
 

Train actual/predicted

Predicted     0     1   All
Actual                     
0.0        1094   233  1327
1.0         223   946  1169
All        1317  1179  2496 

Classification report

              precision    recall  f1-score   support

         0.0       0.83      0.82      0.83      1327
         1.0       0.80      0.81      0.81      1169

    accuracy                           0.82      2496
   macro avg       0.82      0.82      0.82      2496
weighted avg       0.82      0.82    

# Save best XGBoost model

In [39]:
xgboost_model = model_grid.best_estimator_
xgboost_model_path = "./artifacts/lead_model_xgboost.json"
xgboost_model.save_model(xgboost_model_path)

model_results = {
    xgboost_model_path: classification_report(y_train, y_pred_train, output_dict=True)
}

# SKLearn logistic regression

In [40]:
import mlflow.pyfunc

from sklearn.linear_model import LogisticRegression
import os
from sklearn.metrics import cohen_kappa_score, f1_score
import matplotlib.pyplot as plt
import joblib

class lr_wrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model
    
    def predict(self, context, model_input):
        return self.model.predict_proba(model_input)[:, 1]


mlflow.sklearn.autolog(log_input_examples=True, log_models=False)
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

with mlflow.start_run(experiment_id=experiment_id) as run:
    model = LogisticRegression()
    lr_model_path = "./artifacts/lead_model_lr.pkl"

    params = {
              'solver': ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
              'penalty':  ["none", "l1", "l2", "elasticnet"],
              'C' : [100, 10, 1.0, 0.1, 0.01]
    }
    model_grid = RandomizedSearchCV(model, param_distributions= params, verbose=3, n_iter=10, cv=3)
    model_grid.fit(X_train, y_train)

    best_model = model_grid.best_estimator_

    y_pred_train = model_grid.predict(X_train)
    y_pred_test = model_grid.predict(X_test)


    # log artifacts
    mlflow.log_metric('f1_score', f1_score(y_test, y_pred_test))
    mlflow.log_artifacts("artifacts", artifact_path="model")
    mlflow.log_param("data_version", "00000")
    
    # store model for model interpretability
    joblib.dump(value=model, filename=lr_model_path)
        
    # Custom python model for predicting probability 
    mlflow.pyfunc.log_model('model', python_model=lr_wrapper(model))


model_classification_report = classification_report(y_test, y_pred_test, output_dict=True)

best_model_lr_params = model_grid.best_params_

print("Best lr params")
pprint(best_model_lr_params)

print("Accuracy train:", accuracy_score(y_pred_train, y_train ))
print("Accuracy test:", accuracy_score(y_pred_test, y_test))

conf_matrix = confusion_matrix(y_test, y_pred_test)
print("Test actual/predicted\n")
print(pd.crosstab(y_test, y_pred_test, rownames=['Actual'], colnames=['Predicted'], margins=True),'\n')
print("Classification report\n")
print(classification_report(y_test, y_pred_test),'\n')

conf_matrix = confusion_matrix(y_train, y_pred_train)
print("Train actual/predicted\n")
print(pd.crosstab(y_train, y_pred_train, rownames=['Actual'], colnames=['Predicted'], margins=True),'\n')
print("Classification report\n")
print(classification_report(y_train, y_pred_train),'\n')

model_results[lr_model_path] = model_classification_report
print(model_classification_report["weighted avg"]["f1-score"])


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END C=10, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/3] END C=10, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/3] END C=10, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/3] END .C=1.0, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/3] END .C=1.0, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/3] END .C=1.0, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/3] END .......C=100, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/3] END .......C=100, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/3] END .......C=100, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/3] END .....C=10, penalty=l2, solver=saga;, score=0.817 total time=   0.0s
[CV 2/3] END .....C=10, penalty=l2, solver=saga;, score=0.804 total time=   0.0s
[CV 3/3] END .....C=10, penalty=l2, solver

2024/11/27 10:54:39 INFO mlflow.sklearn.utils: Logging the 5 best runs, 5 runs will be omitted.
2024/11/27 10:54:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Best lr params
{'C': 10, 'penalty': 'l2', 'solver': 'saga'}
Accuracy train: 0.8141025641025641
Accuracy test: 0.7845804988662132
Test actual/predicted

Predicted  0.0  1.0  All
Actual                  
0.0        206   29  235
1.0         66  140  206
All        272  169  441 

Classification report

              precision    recall  f1-score   support

         0.0       0.76      0.88      0.81       235
         1.0       0.83      0.68      0.75       206

    accuracy                           0.78       441
   macro avg       0.79      0.78      0.78       441
weighted avg       0.79      0.78      0.78       441
 

Train actual/predicted

Predicted   0.0   1.0   All
Actual                     
0.0        1134   193  1327
1.0         271   898  1169
All        1405  1091  2496 

Classification report

              precision    recall  f1-score   support

         0.0       0.81      0.85      0.83      1327
         1.0       0.82      0.77      0.79      1169

    accuracy    

# Save columns and model results

In [41]:
column_list_path = './artifacts/columns_list.json'
with open(column_list_path, 'w+') as columns_file:
    columns = {'column_names': list(X_train.columns)}
    pprint(columns)
    json.dump(columns, columns_file)

print('Saved column list to ', column_list_path)

model_results_path = "./artifacts/model_results.json"
with open(model_results_path, 'w+') as results_file:
    json.dump(model_results, results_file)

{'column_names': ['purchases',
                  'time_spent',
                  'n_visits',
                  'customer_group_2',
                  'customer_group_3',
                  'customer_group_4',
                  'customer_group_5',
                  'customer_group_6',
                  'customer_group_7',
                  'customer_group_8',
                  'customer_group_9',
                  'onboarding_True']}
Saved column list to  ./artifacts/columns_list.json


# MODEL SELECTION

Model selection involves choosing the most suitable statistical model from a set of candidates. In straightforward cases, this process uses an existing dataset. When candidate models offer comparable predictive or explanatory power, the simplest model is generally the preferred choice.

In [42]:
# Constants used:
current_date = datetime.datetime.now().strftime("%Y_%B_%d")
artifact_path = "model"
model_name = "lead_model"
experiment_name = current_date

# Helper functions

In [43]:
import time
from mlflow.tracking.client import MlflowClient
from mlflow.entities.model_registry.model_version_status import ModelVersionStatus
from mlflow.tracking.client import MlflowClient

def wait_until_ready(model_name, model_version):
    client = MlflowClient()
    for _ in range(10):
        model_version_details = client.get_model_version(
          name=model_name,
          version=model_version,
        )
        status = ModelVersionStatus.from_string(model_version_details.status)
        print(f"Model status: {ModelVersionStatus.to_string(status)}")
        if status == ModelVersionStatus.READY:
            break
        time.sleep(1)


# Getting experiment model results

In [44]:
experiment_ids = [mlflow.get_experiment_by_name(experiment_name).experiment_id]
experiment_ids

['513283018559583915']

In [45]:
experiment_best = mlflow.search_runs(
    experiment_ids=experiment_ids,
    order_by=["metrics.f1_score DESC"],
    max_results=1
).iloc[0]
experiment_best

run_id                                               28e9dba84785460aadab5e5f6f1e4a3d
experiment_id                                                      513283018559583915
status                                                                       FINISHED
artifact_uri                        file:///d:/Projects/SoftwareDev2024/itu-sdse-p...
start_time                                           2024-11-27 09:54:38.865000+00:00
end_time                                             2024-11-27 09:54:49.101000+00:00
metrics.training_recall_score                                                   0.814
metrics.training_log_loss                                                       0.419
metrics.best_cv_score                                                           0.811
metrics.training_f1_score                                                       0.814
metrics.training_score                                                          0.814
metrics.f1_score                                      

In [46]:
import json

with open("./artifacts/model_results.json", "r") as f:
    model_results = json.load(f)
results_df = pd.DataFrame({model: val["weighted avg"] for model, val in model_results.items()}).T
results_df

,precision,recall,f1-score,support
./artifacts/lead_model_xgboost.json,0.817,0.817,0.817,2496.000
./artifacts/lead_model_lr.pkl,0.791,0.785,0.782,441.000


In [47]:
best_model = results_df.sort_values("f1-score", ascending=False).iloc[0].name
print(f"Best model: {best_model}")

Best model: ./artifacts/lead_model_xgboost.json


# Get production model

In [48]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
prod_model = [model for model in client.search_model_versions(f"name='{model_name}'") if dict(model)['current_stage']=='Production']
prod_model_exists = len(prod_model)>0

if prod_model_exists:
    prod_model_version = dict(prod_model[0])['version']
    prod_model_run_id = dict(prod_model[0])['run_id']
    
    print('Production model name: ', model_name)
    print('Production model version:', prod_model_version)
    print('Production model run id:', prod_model_run_id)
    
else:
    print('No model in production')


No model in production


# Compare prod and best trained model

In [49]:
train_model_score = experiment_best["metrics.f1_score"]
model_details = {}
model_status = {}
run_id = None

if prod_model_exists:
    data, details = mlflow.get_run(prod_model_run_id)
    prod_model_score = data[1]["metrics.f1_score"]

    model_status["current"] = train_model_score
    model_status["prod"] = prod_model_score

    if train_model_score>prod_model_score:
        print("Registering new model")
        run_id = experiment_best["run_id"]
else:
    print("No model in production")
    run_id = experiment_best["run_id"]

print(f"Registered model: {run_id}")

No model in production
Registered model: 28e9dba84785460aadab5e5f6f1e4a3d


# Register best model

In [50]:
if run_id is not None:
    print(f'Best model found: {run_id}')

    model_uri = "runs:/{run_id}/{artifact_path}".format(
        run_id=run_id,
        artifact_path=artifact_path
    )
    model_details = mlflow.register_model(model_uri=model_uri, name=model_name)
    wait_until_ready(model_details.name, model_details.version)
    model_details = dict(model_details)
    print(model_details)

Best model found: 28e9dba84785460aadab5e5f6f1e4a3d
Model status: READY
{'aliases': [], 'creation_timestamp': 1732701321234, 'current_stage': 'None', 'description': None, 'last_updated_timestamp': 1732701321234, 'name': 'lead_model', 'run_id': '28e9dba84785460aadab5e5f6f1e4a3d', 'run_link': None, 'source': 'file:///d:/Projects/SoftwareDev2024/itu-sdse-project-main/notebooks/mlruns/513283018559583915/28e9dba84785460aadab5e5f6f1e4a3d/artifacts/model', 'status': 'READY', 'status_message': None, 'tags': {}, 'user_id': None, 'version': 1}


Successfully registered model 'lead_model'.
Created version '1' of model 'lead_model'.


# DEPLOY

A model version can be assigned to one or more stages. MLflow provides predefined stages for common use cases: None, Staging, Production, and Archived. With the necessary permissions, you can transition a model version between stages or request a transition to a different stage.

In [51]:
model_version = 1

# Transition to staging

In [52]:
from mlflow.tracking import MlflowClient

client = MlflowClient()


def wait_for_deployment(model_name, model_version, stage='Staging'):
    status = False
    while not status:
        model_version_details = dict(
            client.get_model_version(name=model_name,version=model_version)
            )
        if model_version_details['current_stage'] == stage:
            print(f'Transition completed to {stage}')
            status = True
            break
        else:
            time.sleep(2)
    return status

model_version_details = dict(client.get_model_version(name=model_name,version=model_version))
model_status = True
if model_version_details['current_stage'] != 'Staging':
    client.transition_model_version_stage(
        name=model_name,
        version=model_version,stage="Staging", 
        archive_existing_versions=True
    )
    model_status = wait_for_deployment(model_name, model_version, 'Staging')
else:
    print('Model already in staging')

Transition completed to Staging
